# Visual Document Classifier v2 training

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-ocr-workshop/blob/master/jupyter/VisualDocumentClassifierTraining/SparkOCRVisualDocumentClassifierv2Training.ipynb)

## Set license and AWS keys

Need to specify:
- secret
- license
- aws credentials

### Option #1 - define in this cell

In [ ]:
import os

secret = ""
version = secret.split("-")[0]

os.environ['JSL_OCR_LICENSE'] = ""
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""

### Option #2 - provide spark_ocr.json file

In [ ]:
import json, os
import sys

if 'google.colab' in sys.modules:
    from google.colab import files

    if 'spark_ocr.json' not in os.listdir():
      license_keys = files.upload()
      os.rename(list(license_keys.keys())[0], 'spark_ocr.json')

with open('spark_ocr.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

## Install Spark-OCR

It is needed only in case of colab. For other environment you should prepare environment appropriately.

In [ ]:
%pip install --upgrade git+https://github.com/JohnSnowLabs/transformers.git@layoutlmv2_onnx
%pip install spark-ocr==$version+spark32 --extra-index-url=https://pypi.johnsnowlabs.com/$secret --upgrade
%pip install sklearn

## Download demo datasets

Here we downloaded demo set. You need to put your images to one folder and prepare labelling txt file as at example.</br>
Instructions here are for the command line, you can also manually download and unzip these files.

In [ ]:
!wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/datasets/visual_doc_classifier/LayoutLM.v2.voc.txt
!wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/datasets/visual_doc_classifier/rvl_cdip_tmp_preprocessed.zip
!wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/datasets/visual_doc_classifier/rvl_cdip_tmp.zip
!unzip rvl_cdip_tmp_preprocessed.zip     
!unzip rvl_cdip_tmp.zip

## Start Spark session with Spark OCR

In [1]:
from sparkocr import start
from pyspark import SparkConf

spark_ocr_jar_path = "../../../../spark-ocr/target/scala-2.12/"
spark = start(jar_path = spark_ocr_jar_path)

spark

Spark version: 3.3.0
Spark NLP version: 4.0.0
Spark NLP for Healthcare version: 4.0.0
Spark OCR version: 4.0.2rc1



## Option #1: Preprocessing your own data

Images for classification should be placed in one folder ("./rvl_cdip_tmp" in this case)

Labels file should be placed to the same folder. File format is the following. One row - one record, file_path and label separated by space like,

```
file1.jpg 1
file2.jpg 2
```

In [2]:
from sparkocr.transformers import *

df = DatasetReader.readDataset("./rvl_cdip_tmp", spark)
display(df.select("content", "act_label").limit(1))

content,act_label
[49 49 2A 00 34 E...,4


### Repartition your data
To better leverage your cluster you may need repartitioning of your input dataframe

In [3]:
df = df.repartition(8)

In [4]:
from sparkocr.transformers import *
from sparkocr.enums import *
from pyspark.ml import PipelineModel

binary_to_image = BinaryToImage()\
    .setOutputCol("image") \
    .setImageType(ImageType.TYPE_3BYTE_BGR)

img_to_hocr = ImageToHocr()\
    .setInputCol("image")\
    .setOutputCol("hocr")\
    .setIgnoreResolution(False)\
    .setOcrParams(["preserve_interword_spaces=0"])

tokenizer = HocrTokenizer()\
    .setInputCol("hocr")\
    .setOutputCol("token")

# OCR pipeline
pipeline1 = PipelineModel(stages=[
    binary_to_image,
    img_to_hocr,
    tokenizer
])

df = pipeline1.transform(df).cache()
df = df.withColumnRenamed("image", "orig_image")
display(df.select("act_label", "pagenum", "exception", "hocr", "token"))

act_label,pagenum,exception,hocr,token
4,0,null,<div class='ocr...,"[{token, 0, 2, in..."


In [5]:
df

orig_image,path,modificationTime,length,act_label,pagenum,exception,hocr,token
{file:/home/jose/...,file:/home/jose/s...,2022-09-06 22:14:30,59658,4,0,null,<div class='ocr...,"[{token, 0, 2, in..."


In [6]:
from sparkocr.utils import get_vocabulary_dict

vocab_file = "LayoutLM.v2.voc.txt"
vocab = get_vocabulary_dict(vocab_file, ",")

doc_class = VisualDocumentClassifierV2() \
    .setInputCols(["token", "orig_image"]) \
    .setOutputCol("label")
doc_class.setVocabulary(vocab)

df = doc_class.getPreprocessedDataset(
  df,
  [1,3,224,224]
  ).cache()

/home/jose/.local/lib/python3.7/site-packages/pyspark/sql/dataframe.py:149: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  "DataFrame.sql_ctx is an internal property, and will be removed "
/home/jose/.local/lib/python3.7/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [7]:
df

orig_image,path,modificationTime,length,act_label,pagenum,hocr,token,input_ids,bbox,image,attention_mask,token_type_ids,exception
{file:/home/jose/...,file:/home/jose/s...,2022-09-06 22:14:30,59658,4,0,<div class='ocr...,"[{token, 0, 2, in...","[101, 7601, 8756,...","[0, 0, 0, 0, 152,...","[255, 255, 255, 2...","[1.0, 1.0, 1.0, 1...","[0, 0, 0, 0, 0, 0...",null


In [9]:
df.select("path", "input_ids", "bbox", "image", "attention_mask", "token_type_ids", "act_label")#.write.parquet("preprocessed_dataset")

path,input_ids,bbox,image,attention_mask,token_type_ids,act_label
file:/home/jose/s...,"[101, 7601, 8756,...","[0, 0, 0, 0, 152,...","[255, 255, 255, 2...","[1.0, 1.0, 1.0, 1...","[0, 0, 0, 0, 0, 0...",4


## Option #2: Use preprocessed datasets
It is possible to load datasets in preprocessed state. You will typically prefer a separate cluster environment to do the preprocessing, as it can take long(a number of hours).
Check this notebook, in the same folder as current one,

Spark-ocr visual doc classifier v2 preprocessing on databricks.ipynb

In [10]:
from sparkocr.transformers import *

df = DatasetReader.readPreprocessedDataset("./rvl_cdip_tmp_preprocessed", spark)
display(df.limit(1))

input_ids,bbox,image,attention_mask,token_type_ids,act_label
"[101, 13169, 1051...","[0, 0, 0, 0, 42, ...","[255, 255, 255, 2...","[1.0, 1.0, 1.0, 1...","[0, 0, 0, 0, 0, 0...",scientific_report


## Training


### Dataframe of preprocessed data
Using either of the above listed options for generating your data, now we will run the training stage.

In [ ]:
from sparkocr.transformers import *

trainer = VisualDocumentClassifierV2()
trainer.set_train_param_model_save_path("new_model")
trainer.set_train_param_vocab_path("LayoutLM.v2.voc.txt")
trainer.set_train_param_spark(spark)
trainer.set_train_param_num_train_epochs(2)
trainer.set_train_param_useGPU(False)

doc_class = trainer.fit(df)

In [ ]:
doc_class